# Objective

Out of the baseline models that were tested, the moving average model performed best with the following stats:

| Metric        | Value
| ------------- |------------- 
| RMSE          | 1.613
| Mean Error    | -0.26
| Median Error  |  0.0
| 95% Error     |  3.0
| 99% Error     |  4.0
| Max Error     |  6.0

The purpose of this notebook is to improve on the moving average model and incorporate the trend and seasonality that was apparent in the time series decomposition and moving average model errors. Ideally, we would like to:

* Provide conservative instead of overly optimistic estimates (non-negative RMSE)
* Reduce the 99% bound

In this notebook, we'll evaluate two different univariate time series models that incorporate trend and seasonality:

1. Holt-Winters (exponential smoothing)
2. Seasonal ARIMA

# Libraries

In [3]:
%load_ext autoreload
%autoreload 2

import sys, os
sys.path.insert(1, os.path.join(sys.path[0], '../src'))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import itertools
import warnings

from app.model.core import score, results_df
from app.model.diagnostics import display_results
from app.feature_data import load_feature_data
from tscv import GapWalkForward

%matplotlib inline

warnings.simplefilter('ignore')

/anaconda3/envs/py37-ds/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/anaconda3/envs/py37-ds/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/anaconda3/envs/py37-ds/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192
  return f(*args, **kwds)
/anaconda3/envs/py37-ds/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/anaconda3/envs/py37-ds/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibi

# Data

In [7]:
X = load_feature_data('../data/processed/weekly-rain-day-counts.csv')
X.head()

,rainEvents
date,
2009-12-27,3
2010-01-03,4
2010-01-10,1
2010-01-17,2
2010-01-24,6


In [8]:
y = X.sort_index()['rainEvents']
y.head()

date
2009-12-27    3
2010-01-03    4
2010-01-10    1
2010-01-17    2
2010-01-24    6
Name: rainEvents, dtype: int64

# Models

Create a function to train and evaluate seasonal models. This iteration of the training and evaluation procedure will use a straightforward walk-forward one-step ahead forecast but we could also consider using nested cross valiation on the tail end of the training set for each fold.

In [55]:
def predict_average(x: pd.Series, horizon: int) -> np.array:
  return x.mean()

In [64]:
horizon = 1

x = y.head()

wf = GapWalkForward(n_splits=len(x)-1, test_size=horizon, gap_size=0)

# Initialize the predictions with an empty value for the first
# value since we need at least one value to train the model
y_hats = { x.index[0]: np.nan }

for train_ix, test_ix in wf.split(x):
  train, test = x.iloc[train_ix], x.iloc[test_ix]
  y_hat = predict_average(train, horizon)
  y_hats[test.index.values[0]] = y_hat
  
y_hats = pd.Series(y_hats)

In [51]:
df = pd.DataFrame()

<bound method Series.set_value of Series([], dtype: float64)>

In [40]:
pd.Series()

{}


In [18]:
len(x)

5

In [11]:
len(errors)

487

In [ ]:
len()

## Holt Winters